In [101]:
import pandas as pd
import networkx as nx
import itertools

In [102]:
df = pd.read_json("rawfilteredbydate.json")

In [103]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [104]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [105]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [78]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle"]

In [92]:
def replaceWord(s):
    for i in range(len(restr)):
        s = s.replace(restr[i],"")
    s = s.replace("and",",")
    return s

In [93]:
"Itzhak Perlman and Catherine Cho perform with PMP Chorus".replace("and",",")

'Itzhak Perlman , Catherine Cho perform with PMP Chorus'

In [94]:
captions = map(lambda x: replaceWord(x),df.caption)

In [95]:
captions[2]

u' Itzhak Perlman , Catherine Cho perform with PMP Chorus '

In [106]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [107]:
import re

In [111]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5:
            name.append(name_temp[0])
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [112]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [113]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [123]:
degrees = degrees.reset_index()

In [124]:
degrees.columns = ["name","number"]

In [125]:
degrees.head()

,name,number
0,PatrickMcMullan,1459
1,Jean Shafiroff,703
2,Patrick McMullan,633
3,Executive,553
4,Mark Gilbertson,549


In [129]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [130]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

[(u'PatrickMcMullan', 1459),
 (u'Jean Shafiroff', 703),
 (u'Patrick McMullan', 633),
 (u'Executive', 553),
 (u'Mark Gilbertson', 549),
 (u'Gillian Miniter', 521),
 (u'Lebenthal', 431),
 (u'Geoffrey Bradfield', 421),
 (u'Yaz Hern', 373),
 (u'Rob Rich', 364),
 (u'Michael', 349),
 (u'Andrew Saffir', 340),
 (u'Sharon Bush', 339),
 (u'Somers Farkas', 337),
 (u'Debbie Bancroft', 333),
 (u'Cutty McGill', 329),
 (u'Kanavos', 321),
 (u'Eleanora Kennedy', 314),
 (u'Alina Cho', 313),
 (u'Clockwise', 310),
 (u'Richard', 299),
 (u'Muffie Potter Aston', 297),
 (u'Dayssi Olarte', 296),
 (u'Bonnie Comley', 293),
 (u'Lucia Hwong Gordon', 276),
 (u'Kamie Lightburn', 276),
 (u'AnnieWatt', 274),
 (u'Annie Watt', 267),
 (u'Mayor Michael Bloomberg', 264),
 (u'Mario Buatta', 264),
 (u'Barbara', 262),
 (u'Bettina Zilkha', 259),
 (u'Foundation', 257),
 (u'Lydia Fenet', 244),
 (u'Allison Aston', 241),
 (u'Stewart Lane', 236),
 (u'Karen LeFrak', 235),
 (u'Elizabeth Stribling', 234),
 (u'Deborah Norville', 231),


In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)